In [1]:
%reload_ext autoreload
%autoreload 2
%cd llama3/llama

/home/tbai4/llama3/llama


/home/tbai4/llama3/.venv/lib/python3.12/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
import os
from llama import Workflow, Llama
from llama.util import load_model_and_tokenizer

os.environ["RANK"] = "0"
os.environ["WORLD_SIZE"] = "1"
os.environ["MASTER_ADDR"] = "localhost"
os.environ["MASTER_PORT"] = "29500"

workflow = Workflow.build(
    ckpt_dir='/scratch4/jeisner1/tjbai/llama_8b',
    tokenizer_path='/scratch4/jeisner1/tjbai/llama_8b/tokenizer.model',
    max_seq_len=8192,
    max_batch_size=1,
    model_parallel_size=1,
    max_nodes=20,
    use_lora=True,
    lora_rank=8,
    lora_alpha=16,
)

> initializing model parallel with size 1
> initializing ddp with size 1
> initializing pipeline with size 1
Converting to LoRA
Loaded in 18.76 seconds


In [3]:
workflow.model.get_trainable_param_percentage()

0.08480376642881861

In [15]:
workflow.reset()
workflow.model.set_adapter_state(enabled=False)

[system] = workflow.insert([
    {
        'messages': [{'role': 'system', 'content': 'Answer the user\'s question please.'}],
        'parent_ids': [],
    },
])

[user_1] = workflow.insert([
    {
        'messages': [{'role': 'user', 'content': 'What is the capital of France?'}],
        'parent_ids': [system['id']],
    },
])

[output], _ = workflow.step(
    [
        {
            'header': ('assistant', None),
            'prefill': '',
            'parent_ids': [system['id'], user_1['id']],
        }
    ]
)

workflow.tokenizer.decode(output)

'The capital of France is Paris.'

In [14]:
workflow.reset()
workflow.model.set_adapter_state(enabled=True)

[system] = workflow.insert([
    {
        'messages': [{'role': 'system', 'content': 'Answer the user\'s question please.'}],
        'parent_ids': [],
    },
])

[user_1] = workflow.insert([
    {
        'messages': [{'role': 'user', 'content': 'What is the capital of France?'}],
        'parent_ids': [system['id']],
    },
])

[output], _ = workflow.step(
    [
        {
            'header': ('assistant', None),
            'prefill': '',
            'parent_ids': [system['id'], user_1['id']],
        }
    ]
)

workflow.tokenizer.decode(output)

'The capital of France is Paris.'

In [16]:
import torch

problem = torch.load('tot_data/problem_0.pt', weights_only=True)

problem['result'].keys()

dict_keys(['proposal_content', 'proposal_tokens', 'proposal_logprobs', 'vote_content', 'vote_tokens', 'vote_logprobs', 'final_content', 'final_tokens', 'final_logprobs', 'votes'])

In [20]:
from llama.workflows.finetune import TotTrainer

trainer = TotTrainer(workflow, branching_factor=8, voters=4)

Training 6.8M / 8.0B parameters


In [37]:
proposal_logprobs, vote_logprobs, final_logprobs = trainer.step(problem)

print(sum(len(p) for p in problem['result']['proposal_tokens']))  # should be off by 8
print(sum(len(v) for v in problem['result']['vote_tokens']))      # should be off by 4
print(len(problem['result']['final_tokens']))                     # should be off by 1

print(proposal_logprobs.shape)
print(vote_logprobs.shape)
print(final_logprobs.shape)

1067
153
116
torch.Size([1, 1075, 128256])
torch.Size([1, 157, 128256])
torch.Size([1, 117, 128256])
